In [1]:
# %load main.py
#!/usr/bin/env python

from params import *
from initialize import *
from utils import get_trajectories, run_setup
import time
from project_utils_2 import *
from manipulation.meshcat_cpp_utils import StartMeshcat
from pydrake.all import Solve
from debug_friction import debug_friction_value, debug_friction_norm
from test import get_vals
import pandas as pd
import altair as alt
import plotly.express as px

def main():
    start_time = time.time()
    result = Solve(prog, initial_guess = initial_guess)
    time_taken = time.time()-start_time
    # solver = SnoptSolver()
    # result = solver.Solve(prog, initial_guess)

    success = result.is_success()
    if success:
        assert True
        print("Total time to complete the trajectory is ",result.get_optimal_cost())
        print("time taken for optimization is ", time_taken)
    else:
        print("Optimization Failed !!!")
        
    if(success and simulate_traj):
        meshcat = StartMeshcat()
        q_traj , q_dot_traj, _, _ = get_trajectories(result.GetSolution(S_dot), result.GetSolution(S_ddot), num_steps = 100)
        diagram, plant, scene_graph, plant_context = MyManipulationStation(mu1 = mu1, mu2 = mu2)
        simulator, plant_context, q_traj, q_dot_traj_ = ExtendedStation(diagram, plant, meshcat, q_traj, q_dot_traj)
        run_setup(simulator, q_traj.end_time())

    return result

#     if(success and debug_fric):
#         lin_acc_val = result.GetSolution(lin_acc)
#         q_ddot_val = result.GetSolution(q_ddot)
#         q_dot_val = result.GetSolution(q_dot)
#         total_force_val = result.GetSolution(total_force)
#         q_val = result.GetSolution(q)
#         debug_friction_value(lin_acc_val, q_ddot_val, q_dot_val, total_force_val, q_val)
        
#         f1 = result.GetSolution(f1)
#         f2 = result.GetSolution(f2)
#         f3 = result.GetSolution(f3)
#         f4 = result.GetSolution(f4)
#         debug_friction_norm(f1, f2, f3, f4)


In [2]:
result = main()

INFO:drake:Meshcat listening for connections at http://localhost:7000


Total time to complete the trajectory is  4.483095317723912
time taken for optimization is  2.1911449432373047


In [3]:

q_traj , q_dot_traj, q_ddot_traj, _ = get_trajectories(result.GetSolution(S_dot), result.GetSolution(S_ddot), num_steps = 100)
data = dataframe(q_traj, q_traj.get_segment_times(), ['q1','q2', 'q3', 'q4', 'q5', 'q6', 'q7'])
alt.vconcat(alt.Chart(data).mark_line().encode(x='t', y='q1').properties(height=80),)



alt.VConcatChart(...)

In [4]:
data = dataframe(q_dot_traj, q_dot_traj.get_segment_times(), ['q1_dot','q2_dot', 'q3_dot', 'q4_dot', 'q5_dot', 'q6_dot', 'q7_dot'])
alt.vconcat(alt.Chart(data).mark_line().encode(x='t', y='q1_dot').properties(height=80),)



alt.VConcatChart(...)

In [5]:
data = dataframe(q_ddot_traj, q_ddot_traj.get_segment_times(), ['q1_ddot','q2_ddot', 'q3_ddot', 'q4_ddot', 'q5_ddot', 'q6_ddot', 'q7_ddot'])
alt.vconcat(alt.Chart(data).mark_line().encode(x='t', y='q1_ddot').properties(height=80),)

alt.VConcatChart(...)